# **Tutorial** - Topic Modeling with BERTopic
(last updated 01-09-2022)

In this tutorial we will be exploring how to use BERTopic to create topics from the well-known 20Newsgroups dataset. The most frequent use-cases and methods are discussed together with important parameters to keep a look out for.


## BERTopic
BERTopic is a topic modeling technique that leverages 🤗 transformers and a custom class-based TF-IDF to create dense clusters allowing for easily interpretable topics whilst keeping important words in the topic descriptions.

<br>

<img src="https://raw.githubusercontent.com/MaartenGr/BERTopic/master/images/logo.png" width="40%">

# Enabling the GPU

First, you'll need to enable GPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down

[Reference](https://colab.research.google.com/notebooks/gpu.ipynb)

# **Installing BERTopic**

We start by installing BERTopic from PyPi:

In [10]:
!pip uninstall numba bertopic umap-learn hdbscan -y
!pip install --upgrade bertopic numba umap-learn hdbscan

Found existing installation: numba 0.63.1
Uninstalling numba-0.63.1:
  Successfully uninstalled numba-0.63.1
Found existing installation: bertopic 0.17.4
Uninstalling bertopic-0.17.4:
  Successfully uninstalled bertopic-0.17.4
Found existing installation: umap-learn 0.5.11
Uninstalling umap-learn-0.5.11:
  Successfully uninstalled umap-learn-0.5.11
Found existing installation: hdbscan 0.8.41
Uninstalling hdbscan-0.8.41:
  Successfully uninstalled hdbscan-0.8.41
  Using cached bertopic-0.17.4-py3-none-any.whl.metadata (24 kB)
  Using cached numba-0.63.1-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (2.9 kB)
  Using cached umap_learn-0.5.11-py3-none-any.whl.metadata (26 kB)
  Using cached hdbscan-0.8.41-cp312-cp312-linux_x86_64.whl
Using cached bertopic-0.17.4-py3-none-any.whl (154 kB)
Using cached numba-0.63.1-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (3.8 MB)
Using cached umap_learn-0.5.11-py3-none-any.whl (90 kB)
ERROR: pip's dependency resolver 

## Restart the Notebook
After installing BERTopic, some packages that were already loaded were updated and in order to correctly use them, we should now restart the notebook.

From the Menu:

Runtime → Restart Runtime

# Data
We use English and Chinese travel reviews datasets scrapping from various platforms including Mafengwo, Xiaohongshu, Qiongyou, TripAdvisor and Reddit which contains roughly 18000 texts.

In [1]:
import pandas as pd

# File paths
MFW_output = '/content/chinese/MFW_output.xlsx'
MFW_youji = '/content/chinese/MFW_youji.xlsx'
qr = '/content/chinese/qr.xlsx'
textqr2_final = '/content/chinese/textqr2_final.xlsx'
tripadvisor_forum_data = '/content/english/tripadvisor_forum_data.xlsx'
xiaohongshu_comments_combined = '/content/chinese/xiaohongshu_comments_combined.csv'
reddit_comments_combined = '/content/english/reddit_comments_combined.csv'
reddit_posts_combined = '/content/english/reddit_posts_combined.csv'
qunar_reviews = '/content/chinese/qunar_reviews.xlsx'
tripadvisor_reviews_data = '/content/english/tripadvisor_reviews_data.csv'
xiecheng = '/content/chinese/xc.xlsx'

# Function to read and extract column data from Excel files
def get_column_data(file_path, column, start_row=1):
    df = pd.read_excel(file_path)
    # Convert column letter to index if needed (e.g., 'A' -> 0, 'H' -> 7)
    if isinstance(column, str):
        column = ord(column.upper()) - ord('A')
    return df.iloc[start_row-1:, column].dropna().astype(str).tolist()

# Function to read and extract column data from csv files
def get_csv_column_data(file_path, column, start_row=1, encoding='utf-8'):
  try:
    df = pd.read_csv(file_path, encoding=encoding)
  except UnicodeDecodeError:
      # Try common alternative encodings if UTF-8 fails
      try:
          df = pd.read_csv(file_path, encoding='latin1')  # Also known as ISO-8859-1
      except:
          df = pd.read_csv(file_path, encoding='gbk')  # Common for Chinese files
    # Convert column letter to index if needed (e.g., 'A' -> 0, 'H' -> 7)
  if isinstance(column, str):
      column = ord(column.upper()) - ord('A')
  return df.iloc[start_row-1:, column].dropna().astype(str).tolist()

# Extract data from each file
mfw_output = get_column_data(MFW_output, 'A', start_row=1)  # A1-Aend
mfw_youji = get_column_data(MFW_youji, 'B', start_row=2)
qr_data = get_column_data(qr, 'A', start_row=2)           # A2-Aend
textqr_data = get_column_data(textqr2_final, 'A', start_row=2)  # A2-Aend
tripadvisor_forum = get_column_data(tripadvisor_forum_data, 'H', start_row=2)  # H2-Hend
xiaohongshu_comments_combined = get_csv_column_data(xiaohongshu_comments_combined, 'F', start_row=2)
reddit_comments_combined = get_csv_column_data(reddit_comments_combined, 'D', start_row=2)
reddit_posts_combined = get_csv_column_data(reddit_posts_combined, 'O', start_row=2)
qunar_reviews_data = get_column_data(qunar_reviews, 'G', start_row=2)
tripadvisor_reviews = get_csv_column_data(tripadvisor_reviews_data, 'E', start_row=2)
xiecheng_data = get_column_data(xiecheng, 'E', start_row = 2)
xiecheng_timestamp = get_column_data(xiecheng, 'F', start_row = 2)

# Concatenate all the data
docs = mfw_output + mfw_youji + qr_data + textqr_data + tripadvisor_forum + xiaohongshu_comments_combined + reddit_comments_combined + reddit_posts_combined + qunar_reviews_data + tripadvisor_reviews + xiecheng_data



# concatenate all chinese data
docs_chinese = mfw_output + mfw_youji + qr_data + textqr_data + xiaohongshu_comments_combined + qunar_reviews_data + xiecheng_data

# concatenate all english data
docs_english = reddit_comments_combined + reddit_posts_combined + tripadvisor_forum + tripadvisor_reviews

# 描述澳门形象和凝视焦点的中文数据
docs_chinese_focus = mfw_youji + xiaohongshu_comments_combined + qunar_reviews_data + xiecheng_data

# 描述访澳游客核心需求的中文数据
docs_chinese_demand = mfw_output + qr_data + textqr_data + xiaohongshu_comments_combined

# 描述澳门形象和凝视焦点的英文数据
docs_english_focus = reddit_posts_combined + tripadvisor_reviews

# 描述访澳游客核心需求的英文数据
docs_english_demand = reddit_comments_combined + tripadvisor_forum



In [2]:
print(f'total number of documents: {len(docs)}')
print(f'number of english documents: {len(docs_english)}')
print(f'number of chinese documents: {len(docs_chinese)}')
print(f'number of chinese_focus documents: {len(docs_chinese_focus)}')
print(f'number of chinese_demand documents: {len(docs_chinese_demand)}')
print(f'number of english_focus documents: {len(docs_english_focus)}')
print(f'number of english_demand documents: {len(docs_english_demand)}')

total number of documents: 58735
number of english documents: 30512
number of chinese documents: 28223
number of chinese_focus documents: 23710
number of chinese_demand documents: 7540
number of english_focus documents: 24045
number of english_demand documents: 6467


In [3]:
!pip install emoji
!pip install opencc
!pip install pycountry

In [4]:
# 数据清洗

import re
import emoji
import opencc

def remove_emojis(text):
    return emoji.replace_emoji(text, replace='')

converter = opencc.OpenCC('t2s.json')

# Clean the text
def clean_text(text):
    # remove URLs first (both <http...> and www formats)
    text = re.sub(r'<https?:\/\/[^\s>]+>|www\.[^\s>]+', '', text)
    # remove symbols
    text = re.sub(r'[\[\]「」★■【】*#@&$%\^\*\-\+=\|～\\/~\u3000]', '', text)
    # Then remove emojis
    text = remove_emojis(text)

    # Add removal of specific words
    #words_to_remove = ["攻略首页", "非常抱歉，您访问的页面不存在。", "去哪儿旅行", "首页", "攻略", "适老化及无障碍", "请登录或免费注册", "消息", "查看订单联系客服", "NEW一日游", "团购", "邮轮", "度假", "门票", "火车票", "当地人", "汽车票", "更多", "攻略库", "目的地", "创建行程", "回发表游记", "创作者平台", "搜目的地/攻略", "旅行》目的地》中国>澳门澳门景点》", "评论详情", "更多周边景点", "意见反馈", "纠错", "反馈", "目的地分类导航", "澳门热门景点", "大三巴牌坊", "澳门塔", "岗顶前地", "龙环葡韵住宅式", "主教山小堂", "澳门市政署", "亚婆井前地", "大炮台", "妈阁庙", "澳门博物馆", "玫瑰圣母堂", "圣若瑟修院藏珍馆", "澳门渔人码头", "金莲花广场", "卢家大屋", "圣母诞辰主教座堂圣老楞佐堂", "港务局", "郑家大屋", "澳门大赛车博物馆", "中国热门城市", "北京旅游", "厦门旅游", "上海旅游", "杭州旅游", "南京旅游", "成都旅游", "丽江旅游", "香港旅游", "澳门旅游", "台北旅游", "三亚旅游", "西安旅游", "桂林旅游", "重庆旅游", "青岛旅游", "苏州旅游", "大连旅游", "广州旅游", "深圳旅游", "张家界旅游", "12025热门目的地推荐", "天津旅游", "泰国旅游", "英国旅游", "法国旅游", "意大利旅游", "美国旅游", "加拿大旅游", "澳大利亚旅游", "埃及旅游", "韩国旅游", "马尔代夫旅游", "德国旅游", "土耳其旅游", "新西兰旅游", "肯尼亚旅游", "南非旅游", "印度尼西亚旅游", "菲律宾旅游", "日本旅游", "瑞士旅游", "Qunar.com", "业务合作", "加入我们", "严重违规", "失信专项整治", "举报", "安全中心", "星骆驼公益", "AboutUs", "Trip.com", "Group", "Copyright@2021Qunar.com", "京公网安备", "11010802030542", "京ICP备", "05021087号", "京ICP证", "060856号", "营业执照信息", "互联网药品信息服务资格证", "（京）-非经营性-2016-0110C去哪儿", "投诉", "咨询热线电话", "95117", "投诉邮箱", "tousu@qunar.com", "全国旅游投诉热线", "12345", "未成年人", "违法和不良信息", "算法推荐", "举报电话", "010-59606977", "举报邮箱", "信用中国", "Osecure", "网上有害信息", "举报专区", "旅行》目的地》中国"]  # Add more words as needed
    words_for_removal = ["阅读全部"]
    for word in words_for_removal:
        text = text.replace(word, '')

    # 繁体转简体
    text = converter.convert(text)

    return text

# Apply to all docs
docs = [clean_text(doc) for doc in docs]
docs_chinese = [clean_text(doc) for doc in docs_chinese]
docs_chinese_focus = [clean_text(doc) for doc in docs_chinese_focus]
docs_chinese_demand = [clean_text(doc) for doc in docs_chinese_demand]
docs_english = [clean_text(doc) for doc in docs_english]
docs_english_focus = [clean_text(doc) for doc in docs_english_focus]
docs_english_demand = [clean_text(doc) for doc in docs_english_demand]

In [ ]:
# randomly inspect documents after cleansing
import pandas as pd

docs_inspect = []
for i in range(0, len(docs), 100):
  docs_inspect.append(docs[i])

pd.DataFrame(docs_inspect, columns=['Documents']).to_excel('documents_for_inspect.xlsx', index=False)

In [5]:
### Training data statistics

# total word count
def count_words(doc):
    """
    Count the total words in a document containing English or Chinese text.
    For English: splits on whitespace and punctuation
    For Chinese: counts each character as a word (common approach)
    Handles mixed English-Chinese documents correctly.
    """
    # Chinese characters (Unicode blocks)
    chinese_pattern = re.compile(r'[\u4e00-\u9fff\u3400-\u4dbf\U00020000-\U0002a6df\U0002a700-\U0002b73f\U0002b740-\U0002b81f\U0002b820-\U0002ceaf]+')

    # Count Chinese characters (each counts as 1 word)
    chinese_words = sum(len(match) for match in chinese_pattern.findall(doc))

    # Count English words (split by non-word characters)
    english_words = len(re.findall(r'[a-zA-Z]+', doc))

    return chinese_words + english_words


total_word_count = 0

for doc in docs:
  word_count = count_words(doc)
  total_word_count += word_count

print(f'total word count of all documents is: ', total_word_count)

total word count of all documents is:  4598410


In [ ]:
#count region number
import pycountry
from collections import defaultdict

tripadvisor_forum_region = get_column_data(tripadvisor_forum_data, 'E', start_row=2)
tripadvisor_reviews_region = get_csv_column_data(tripadvisor_reviews_data, 'C', start_row=2)
xiecheng

print(tripadvisor_forum_region, tripadvisor_reviews_region)


# 国家/地区标准化映射表
COUNTRY_MAPPING = {
    # 处理特殊缩写和别名
    "AB": "Canada",          # 加拿大阿尔伯塔省
    "AK": "United States",    # 美国阿拉斯加州
    "AZ": "United States",    # 美国亚利桑那州
    "BC": "Canada",           # 加拿大不列颠哥伦比亚省
    "UK": "United Kingdom",
    "Turkiye": "Turkey",
    # 处理城市归并（可选）
    "Bacolod": "Philippines",
    "Batangas": "Philippines",
    # 处理空值和无效值
    "": "Unknown",
    "Asia": "Unknown"
}

def standardize_country(raw_name):
    # 清洗数据：去除空格和特殊符号
    cleaned = raw_name.strip().split('...')[0].split(',')[-1].strip()

    # 优先检查预设映射表
    if cleaned in COUNTRY_MAPPING:
        return COUNTRY_MAPPING[cleaned]

    # 使用pycountry匹配标准国家名
    try:
        return pycountry.countries.search_fuzzy(cleaned)[0].name
    except LookupError:
        return "Unknown"


# Your input list
locations = tripadvisor_forum_region + tripadvisor_reviews_region

# 标准化并计数
country_counts = defaultdict(int)
for loc in locations:
    country = standardize_country(loc)
    country_counts[country] += 1

# 过滤无效值并排序
valid_counts = {k: v for k, v in sorted(country_counts.items())
                if k != "Unknown" and v > 0}

print(f"实际国家/地区数量: {len(valid_counts)}")
for country, count in valid_counts.items():
    print(f"{country}: {count}")

['Bangkok, Thailand', 'New Delhi, India', 'Manila, Philippines', 'World', 'Richmond, New...', 'Canada', 'Indonesia', 'Istanbul, Turkiye', 'Mumbai, India', 'Hong Kong, China', 'Murray Bridge...', 'Marietta, Georgia', 'Melbourne, Australia', 'Gavle, Sweden', 'Gavle, Sweden', 'Gavle, Sweden', 'Sydney, Australia', 'Perth, Australia', 'Hong Kong, China', 'Yangon (Rangoon...', 'Singapore', 'Singapore', 'Bangkok, Thailand', 'Bangkok, Thailand', 'Singapore, Singapore', 'Istanbul, Turkiye', 'Singapore', 'london', 'Sydney, Australia', 'Philippines', 'Canada', 'Ontario, Canada', 'london', 'Indonesia', 'Hong Kong, China', 'Bideford', 'Indonesia', 'Hong Kong, China', 'Kathmandu, Nepal', 'Southampton, United...', 'Bangkok, Thailand', 'Palm Beach...', 'Palm Beach...', 'Mexico City, Mexico', 'Brisbane, Australia', 'Marietta, Georgia', 'Singapore, Singapore', 'Warrington, United...', 'Hong Kong, China', 'Ambala, India', 'Singapore, Singapore', 'Bialystok, Poland', 'Lincoln, California', 'Singapore, Sin

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-15-fc1d2e919c71>", line 49, in <cell line: 0>
    country = standardize_country(loc)
              ^^^^^^^^^^^^^^^^^^^^^^^^
  File "<ipython-input-15-fc1d2e919c71>", line 38, in standardize_country
    return pycountry.countries.search_fuzzy(cleaned)[0].name
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pycountry/__init__.py", line 75, in search_fuzzy
    match_subdivions = pycountry.Subdivisions.match(
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pycountry/__init__.py", line None, in match
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPyt

In [ ]:
# temporal range

from datetime import datetime
from dateutil.parser import parse

def find_date_range(date_strings):
    dates = []
    for date_str in date_strings:
        try:
            # Parse the date string automatically (handles most common formats)
            date = parse(date_str)
            dates.append(date)
        except ValueError:
            print(f"Warning: Could not parse date: {date_str}")
            continue

    if not dates:
        return None, None

    return min(dates), max(dates)

# Example usage
qunar_reviews_time = get_column_data(qunar_reviews, 'C', start_row=2)
tripadvisor_reviews_time = get_csv_column_data(tripadvisor_reviews_data, 'F', start_row=2)
tripadvisor_forum_time = get_column_data(tripadvisor_forum_data, 'C', start_row=2)
date_list = qunar_reviews_time + tripadvisor_reviews_time + tripadvisor_forum_time

print(date_list)

earliest, latest = find_date_range(date_list)

print(f"Earliest date: {earliest.strftime('%Y-%m-%d')}")
print(f"Most recent date: {latest.strftime('%Y-%m-%d')}")

['2015-11-08', '2018-01-10', '2019-10-09', '2018-08-19', '2018-05-19', '2018-01-12', '2017-12-20', '2015-11-10', '2015-06-11', '2019-12-30', '2019-10-17', '2019-09-24', '2018-11-08', '2018-08-20', '2018-02-22', '2018-02-16', '2017-02-24', '2016-09-30发表于游记台湾自由行，2016不一样的春节', '2015-11-30', '2015-09-13', '2015-08-30', '2015-08-19', '2015-04-10', '2024-02-08', '2019-09-24', '2019-09-24', '2019-09-23', '2019-09-23', '2019-04-14', '2019-01-29', '2019-01-29', '2018-11-26', '2018-11-25', '2018-11-25', '2018-11-08', '2018-05-06', '2018-03-24', '2018-02-23', '2018-02-22', '2018-02-14', '2018-01-24', '2018-01-24', '2018-01-18', '2018-01-18', '2017-12-13发表于游记香港&澳门，遇见不一样的你', '2017-11-27发表于游记惊世巨制「水舞间」——澳门畅游体验行【完结】', '2017-10-18', '2017-10-16', '2017-07-17发表于游记赏味记 当生活一塌糊涂的时候，我跑去了澳门吃叉烧蛋', '2017-05-19发表于游记领着5个老人，1天澳门暴走大计划', '2017-02-01', '2017-01-14', '2017-01-12发表于游记【我们出发吧】送我四天青春，带你发现不一样的澳门', '2017-01-09', '2016-12-20发表于游记繁华里外，澳门的A面与B面', '2016-06-13', '2016-05-13', '2015-11-24', '2015-11-19', '2015-11-

In [6]:
# inspect documents
print(docs[14790])

拿好行李后，问去银河的班车在哪里乘，结果答案是：我们在南门，班车在北门就这样我们来来回回、反反复复的对穿赌场。好不容易到了银河工作人员告诉我们去新马路的车刚刚开走了，而且是最后一班，晕啊这下肿么办
要买的东西走断腿都没兜到，要搭的班车有没乘到末班，诸事不顺啊
后来问了工作人员还有什么班车可以过去的，因为银河、威尼斯、金沙、新濠是在这个岛的，而主要景点和酒店是在另一个岛上的...



# **Topic Modeling**

In this example, we will go through the main components of BERTopic and the steps necessary to create a strong topic model.




## Training

We start by instantiating BERTopic. We set language to `english` when documents are in the English language. When the documents are in chinese or other languages, we use `language="multilingual"` instead.

We will also calculate the topic probabilities. However, this can slow down BERTopic significantly at large amounts of data (>100_000 documents). It is advised to turn this off if you want to speed up the model.


In [27]:

from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance, OpenAI, PartOfSpeech
import openai
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder


# pre-calculate embeddings
# embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
# embeddings = embedding_model.encode(docs_chinese, show_progress_bar=True)

# # finetune dimensionality reduction model
# umap_model = UMAP(n_neighbors=15, n_components=5, metric='cosine', low_memory=False, random_state=42)

# # finetune clustering model
# hdbscan_model = HDBSCAN(min_cluster_size=25, metric='euclidean', prediction_data=True)

prompt_cn_focus = """
基于这些关键词：[KEYWORDS]
和基于这些代表文件：[DOCUMENTS]

请用中文创建不超过7个字的主题标签，要求聚焦澳门形象和访澳旅客的凝视焦点。
"""

prompt_en_focus = """
Based on these keywords: [KEYWORDS]
And these documents: [DOCUMENTS]

Create a technical label (max 5 words and in English) for this topic. This techinical label should describe either Macau's visitors' attention focus or Macau's image and identity
"""

prompt_cn_demand = """
基于这些关键词：[KEYWORDS]
和基于这些代表文件：[DOCUMENTS]

请用中文创建不超过7个字的主题标签，要求聚焦访澳旅客的核心需求。
"""

prompt_en_demand = """
Based on these keywords: [KEYWORDS]
And these documents: [DOCUMENTS]

Create a technical label (max 5 words and in English) for this topic. This techinical label should describe Macau's visitors' core demand.
"""

client = openai.OpenAI(api_key="")
representation_model0 = None
representation_model1 = KeyBERTInspired()
representation_model2 = MaximalMarginalRelevance(diversity=0)
representation_model3 = OpenAI(
    client,
    model="gpt-4.1",
    prompt=prompt_cn_focus, #按照情况修改
    nr_docs=4,
    doc_length=100,
    diversity=0.2,
    tokenizer= "whitespace",
    delay_in_seconds=2
)

seed_topic_list = [["文化环境"],["一般基础设施"],["一般氛围"],["社会环境"],["自然环境"],["旅游基础设施"],["高档氛围"]]
#seed_topic_list = [["Cultural Environment"],["General Infrastructure"],["General Atmosphere"],["Social Environment"],["Natural Environment"],["Tourism Infrastructure"],["Upscale Atmosphere"]]

topic_model = BERTopic(language="chinese (simplified)", seed_topic_list=seed_topic_list, top_n_words=10, min_topic_size=10, representation_model=representation_model2, calculate_probabilities=True, verbose=True) #按文本语言修改模型语言和min_topic_size
#topic_model = BERTopic(language="english", seed_topic_list=seed_topic_list, top_n_words=10, min_topic_size=10, representation_model=representation_model2, calculate_probabilities=True, verbose=True) #按文本语言修改模型语言和min_topic_size

topics, probs = topic_model.fit_transform(docs_chinese_focus) #修改文本



2026-01-23 07:16:28,108 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/741 [00:00<?, ?it/s]

2026-01-23 07:16:54,361 - BERTopic - Embedding - Completed ✓
2026-01-23 07:16:54,362 - BERTopic - Guided - Find embeddings highly related to seeded topics.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2026-01-23 07:16:54,546 - BERTopic - Guided - Completed ✓
2026-01-23 07:16:54,548 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2026-01-23 07:17:18,930 - BERTopic - Dimensionality - Completed ✓
2026-01-23 07:17:18,931 - BERTopic - Cluster - Start clustering the reduced embeddings
2026-01-23 07:18:50,723 - BERTopic - Cluster - Completed ✓
2026-01-23 07:18:50,734 - BERTopic - Representation - Fine-tuning topics using representation models.
2026-01-23 07:18:59,419 - BERTopic - Representation - Completed ✓


**NOTE**: Use `language="multilingual"` to select a model that support 50+ languages.

## Extracting Topics
After fitting our model, we can start by looking at the results. Typically, we look at the most frequent topics first as they best represent the collection of documents.

In [28]:
import jieba

professional_terms = ["圣安多尼", "聖安多尼", "何超琼", "梁安琪", "花王堂", "港珠澳大桥", "路易十七", "权志龙", "威尼斯人", "伦敦人", "巴黎人", "王楚钦", "特别行政", "花王堂区", "风顺堂区", "花地玛堂区", "望德堂区", "圣方济各堂区", "港澳通行证", "疯堂斜巷"]
for term in professional_terms:
    jieba.add_word(term, freq=10000, tag='nz')  # 高频+专有名词标签

def scel_to_txt(scel_file, txt_file):
    with open(scel_file, 'rb') as f_scel, open(txt_file, 'w', encoding='utf-8') as f_txt:
        f_scel.read(0x154)  # Skip header
        while True:
            word_bytes = f_scel.read(2)
            if not word_bytes:
                break
            word = word_bytes.decode('utf-16le', errors='ignore')
            f_scel.read(2)  # Skip frequency/other data
            f_txt.write(word + '\n')
    print(f"Converted {scel_file} to {txt_file}")


scel_to_txt("/content/sougou_travel_words.scel", "output.txt") # sougou_travel_words.scel is available in the github repo 'jqiu19/Topic_Modelling'
jieba.load_userdict("output.txt")

# finetune vectorizer to improve topic representation
stop_words_cn = [
    "$", "0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "?", "_", "“", "”", "、", "。", "《", "》", "里面", "终于", "一座", "一", "一些", "一何",
    "一切", "一则", "一方面", "一旦", "一来", "一样", "一般", "一转眼", "万一", "上", "上下", "下", "不", "不仅", "不但",
    "不光", "不单", "不只", "不外乎", "不如", "不妨", "不尽", "不尽然", "不得", "不怕", "不惟", "不成", "不拘", "不料",
    "不是", "不比", "不然", "不特", "不独", "不管", "不至于", "不若", "不论", "不过", "不问", "与", "与其", "与其说",
    "与否", "与此同时", "且", "且不说", "且说", "两者", "个", "个别", "临", "为", "为了", "为什么", "为何", "为止",
    "为此", "为着", "乃", "乃至", "乃至于", "么", "之", "之一", "之所以", "之类", "乌乎", "乎", "乘", "也", "也好",
    "也罢", "了", "二来", "于", "于是", "于是乎", "云云", "云尔", "些", "亦", "人", "人们", "人家", "什么", "什么样",
    "今", "介于", "仍", "仍旧", "从", "从此", "从而", "他", "他人", "他们", "以", "以上", "以为", "以便", "以免", "以及",
    "以故", "以期", "以来", "以至", "建于", "以至于", "以致", "们", "任", "任何", "任凭", "似的", "但", "但凡", "但是", "何",
    "何以", "何况", "何处", "何时", "余外", "作为", "你", "你们", "使", "使得", "例如", "依", "依据", "依照", "便于",
    "俺", "俺们", "倘", "倘使", "倘或", "倘然", "倘若", "借", "假使", "假如", "假若", "傥然", "像", "儿", "先不先",
    "光是", "全体", "全部", "兮", "关于", "其", "其一", "其中", "其二", "其他", "其余", "其它", "其次", "具体地说",
    "具体说来", "兼之", "内", "再", "再其次", "再则", "再有", "再者", "再者说", "再说", "冒", "冲", "况且", "几", "几时",
    "凡", "凡是", "凭", "凭借", "出于", "出来", "分别", "则", "则甚", "别", "别人", "别处", "别是", "别的", "别管", "别说",
    "到", "前后", "前此", "前者", "加之", "加以", "即", "即令", "即使", "即便", "即如", "即或", "即若", "却", "去", "又",
    "又及", "及", "及其", "及至", "反之", "反而", "反过来", "反过来说", "受到", "另", "另一方面", "另外", "另悉", "只",
    "只当", "只怕", "只是", "只有", "只消", "只要", "只限", "叫", "叮咚", "可", "可以", "可是", "可见", "各", "各个",
    "各位", "各种", "各自", "同", "同时", "后", "后者", "向", "向使", "向着", "吓", "吗", "否则", "吧", "吧哒", "吱", "呀",
    "呃", "呕", "呗", "呜", "呜呼", "呢", "呵", "呵呵", "呸", "呼哧", "咋", "和", "咚", "咦", "咧", "咱", "咱们", "咳", "哇",
    "哈", "哈哈", "哉", "哎", "哎呀", "哎哟", "哗", "哟", "哦", "哩", "哪", "哪个", "哪些", "哪儿", "哪天", "哪年", "哪怕",
    "哪样", "哪边", "哪里", "哼", "哼唷", "唉", "唯有", "啊", "啐", "啥", "啦", "啪达", "啷当", "喂", "喏", "喔唷", "喽",
    "嗡", "嗡嗡", "嗬", "嗯", "嗳", "嘎", "嘎登", "嘘", "嘛", "嘻", "嘿", "嘿嘿", "因", "因为", "因了", "因此", "因着",
    "因而", "固然", "在", "在下", "在于", "地", "基于", "处在", "多", "多么", "多少", "大", "大家", "她", "她们", "好", "如",
    "如上", "如上所述", "如下", "如何", "如其", "如同", "如是", "如果", "如此", "如若", "始而", "孰料", "孰知", "宁",
    "宁可", "宁愿", "宁肯", "它", "它们", "对", "对于", "对待", "对方", "对比", "将", "小", "尔", "尔后", "尔尔", "尚且",
    "就", "就是", "就是了", "就是说", "就算", "就要", "尽", "尽管", "尽管如此", "岂但", "己", "已", "已矣", "巴", "巴巴",
    "并", "并且", "并非", "庶乎", "庶几", "开外", "开始", "归", "归齐", "当", "当地", "当然", "当着", "彼", "彼时", "彼此",
    "往", "待", "很", "得", "得了", "怎", "怎么", "怎么办", "怎么样", "怎奈", "怎样", "总之", "总的来看", "总的来说",
    "总的说来", "总而言之", "恰恰相反", "您", "惟其", "慢说", "我", "我们", "或", "或则", "或是", "或曰", "或者", "截至",
    "所", "所以", "所在", "所幸", "所有", "才", "才能", "打", "打从", "把", "抑或", "拿", "按", "按照", "换句话说", "换言之",
    "据", "据此", "接着", "故", "故此", "故而", "旁人", "无", "无宁", "无论", "既", "既往", "既是", "既然", "时候", "是",
    "是以", "是的", "曾", "替", "替代", "最", "有", "有些", "有关", "有及", "有时", "有的", "望", "朝", "朝着", "本", "本人",
    "本地", "本着", "本身", "来", "来着", "来自", "来说", "极了", "果然", "果真", "某", "某个", "某些", "某某", "根据", "欤",
    "正值", "正如", "正巧", "正是", "此", "此地", "此处", "此外", "此时", "此次", "此间", "毋宁", "每", "每当", "比", "比及",
    "比如", "比方", "没奈何", "沿", "沿着", "漫说", "焉", "然则", "然后", "然而", "照", "照着", "犹且", "犹自", "甚且",
    "甚么", "甚或", "甚而", "甚至", "甚至于", "用", "用来", "由", "由于", "由是", "由此", "由此可见", "的", "的确", "的话",
    "直到", "相对而言", "省得", "看", "眨眼", "着", "着呢", "矣", "矣乎", "矣哉", "离", "竟而", "第", "等", "等到", "等等",
    "简言之", "管", "类如", "紧接着", "纵", "纵令", "纵使", "纵然", "经", "经过", "结果", "给", "继之", "继后", "继而",
    "综上所述", "罢了", "者", "而", "而且", "而况", "而后", "而外", "而已", "而是", "而言", "能", "能否", "腾", "自",
    "自个儿", "自从", "自各儿", "自后", "自家", "自己", "自打", "自身", "至", "至于", "至今", "至若", "致", "般的", "若",
    "若夫", "若是", "若果", "若非", "莫不然", "莫如", "莫若", "虽", "虽则", "虽然", "虽说", "被", "要", "要不", "要不是",
    "要不然", "要么", "要是", "譬喻", "譬如", "让", "许多", "论", "设使", "设或", "设若", "诚如", "诚然", "该", "说来",
    "诸", "诸位", "诸如", "谁", "谁人", "谁料", "谁知", "贼死", "赖以", "赶", "起", "起见", "趁", "趁着", "越是", "距", "跟",
    "较", "较之", "边", "过", "还", "还是", "还有", "还要", "这", "这一来", "这个", "这么", "这么些", "这么样", "这么点儿",
    "这些", "这会儿", "这儿", "这就是说", "这时", "这样", "这次", "这般", "一个", "这边", "这里", "进而", "连", "连同", "逐步",
    "通过", "遵循", "遵照", "那", "那个", "那么", "那么些", "那么样", "那些", "那会儿", "那儿", "那时", "那样", "那般",
    "那边", "那里", "都", "鄙人", "鉴于", "针对", "阿", "除", "除了", "除外", "除开", "除此之外", "除非", "随", "随后",
    "随时", "随着", "难道说", "非但", "非徒", "非特", "非独", "靠", "顺", "顺着", "首先", "...", "一起", "真的", "位于", "害羞", "澳门", "澳門", "！", "，", "：", "；", "？", "没有", "人人", "话题", "r"
] # 中文停用词，即中文段内出现次数多但对文本意思无实际贡献的词

def tokenize_zh(text):
    words = jieba.lcut(text)

    # Filter out unwanted tokens
    words = [word.strip() for word in words
             if word.strip()
             and len(word.strip()) > 1  # Remove single characters
             and not word.isspace()
             and not word.isdigit()
             and not re.search(r'[a-zA-Z]', word)] # Remove english characters
    return words

vectorizer_model = CountVectorizer(ngram_range=(1, 3), stop_words=stop_words_cn, min_df=2, max_df=0.9, tokenizer=tokenize_zh, token_pattern=None) # min_df=6 意思是该词必须出现在至少6个文档中才能被考虑为主题词, max_df=0.9忽略在90%的文档中都出现了的词
#按文本语言修改stop_words，和tokenizer

#vectorizer_model = CountVectorizer(ngram_range=(1, 3), stop_words=None, min_df=2, max_df=0.9, tokenizer=tokenize, token_pattern=None) # min_df=6 意思是该词必须出现在至少6个文档中才能被考虑为主题词, max_df=0.9忽略在90%的文档中都出现了的词
#按文本语言修改stop_words，和tokenizer
topic_model.update_topics(docs_chinese_focus, vectorizer_model=vectorizer_model, top_n_words=10) #修改文本
#topic_model.update_topics(docs_english_focus, vectorizer_model=vectorizer_model, top_n_words=10) #修改文本

Converted /content/sougou_travel_words.scel to output.txt


In [29]:
from pathlib import Path
import pandas as pd
from collections import defaultdict

freq = topic_model.get_topic_info();

def postprocessing(df):
    # Convert all representations to component lists
    all_rows = [words.copy() for words in df['Representation']]
    global_components = set()

    for i in range(len(all_rows)):
        current_row = all_rows[i]
        filtered_words = []
        words_kept = 0

        for word in current_row:
            components = word.split()

            # Check against all previously seen components (current row + global)
            duplicate = any(comp in global_components for comp in components)

            if not duplicate:
                filtered_words.append(word)
                global_components.update(components)
                words_kept += 1

                # Stop current row if we reach 10 words
                if words_kept >= 15:
                    break

        # Update the row (may have less than 10 words at this point)
        all_rows[i] = filtered_words

    # Final truncation to ensure exactly 10 words per row
    for i in range(len(all_rows)):
        all_rows[i] = all_rows[i][:15]

    # Update dataframe
    df['Representation'] = all_rows
    return df


#freq = postprocessing(freq)
freq.head(100).to_excel("/content/topic_frequencies_list.xlsx", index=False)


In [30]:
# Step 2: Reduce to exactly 15 topics
topic_model_reduced = topic_model.reduce_topics(docs_chinese_focus, nr_topics=16)
topics_reduced, probs_reduced = topic_model_reduced.transform(docs_chinese_focus)

2026-01-23 07:22:31,510 - BERTopic - Topic reduction - Reducing number of topics
2026-01-23 07:22:31,555 - BERTopic - Representation - Fine-tuning topics using representation models.
2026-01-23 07:22:42,719 - BERTopic - Representation - Completed ✓
2026-01-23 07:22:42,726 - BERTopic - Topic reduction - Reduced number of topics from 260 to 16


Batches:   0%|          | 0/741 [00:00<?, ?it/s]

2026-01-23 07:23:08,232 - BERTopic - Dimensionality - Reducing dimensionality of input embeddings.
2026-01-23 07:23:15,861 - BERTopic - Dimensionality - Completed ✓
2026-01-23 07:23:15,863 - BERTopic - Clustering - Approximating new points with `hdbscan_model`
2026-01-23 07:23:17,304 - BERTopic - Probabilities - Start calculation of probabilities with HDBSCAN
2026-01-23 07:24:52,957 - BERTopic - Probabilities - Completed ✓
2026-01-23 07:24:52,958 - BERTopic - Cluster - Completed ✓


In [31]:
freq = topic_model.get_topic_info();

def postprocessing(df):
    # Convert all representations to component lists
    all_rows = [words.copy() for words in df['Representation']]
    global_components = set()

    for i in range(len(all_rows)):
        current_row = all_rows[i]
        filtered_words = []
        words_kept = 0

        for word in current_row:
            components = word.split()

            # Check against all previously seen components (current row + global)
            duplicate = any(comp in global_components for comp in components)

            if not duplicate:
                filtered_words.append(word)
                global_components.update(components)
                words_kept += 1

            # Stop current row if we reach 10 words
            if words_kept >= 15:
                break

        # Update the row (may have less than 10 words at this point)
        all_rows[i] = filtered_words

    # Final truncation to ensure exactly 10 words per row
    for i in range(len(all_rows)):
        all_rows[i] = all_rows[i][:15]

    # Update dataframe
    df['Representation'] = all_rows
    return df


#freq = postprocessing(freq)
freq.head(18).to_excel("/content/topic_frequencies_list_reduced.xlsx", index=False)


[('酒店', np.float64(0.015794855617877846)), ('建筑', np.float64(0.014745948348970268)), ('教堂', np.float64(0.014580848859784851)), ('历史', np.float64(0.011622230190892311)), ('博物馆', np.float64(0.009705067700597128)), ('码头', np.float64(0.00899664720879912)), ('景色', np.float64(0.008926034683663018)), ('拍照', np.float64(0.008902108591815238)), ('旅游', np.float64(0.008396801633723364)), ('大三巴牌坊', np.float64(0.00834930939246645))]


In [46]:
topic_15_words = topic_model.get_topic(15)
print(topic_15_words)

False


In [ ]:
# generate topic label using gpt-4.1
topic_model.update_topics(docs_chinese_focus, representation_model=representation_model3)#修改文本

freq = topic_model.get_topic_info();
freq.head(100)
freq.head(100).to_excel("/content/topic_frequencies_label.xlsx", index=False)

100%|██████████| 67/67 [03:37<00:00,  3.24s/it]


In [ ]:
# save model
from datetime import datetime

embedding_model = "sentence-transformers/all-MiniLM-L6-v2"
topic_model.save("/content/", serialization="safetensors", save_ctfidf=True, save_embedding_model=embedding_model)


-1 refers to all outliers and should typically be ignored. Next, let's take a look at a frequent topic that were generated:

In [ ]:
topic_model.get_topic(0)  # Select the most frequent topic

[('macau', np.float64(0.05101931384878174)),
 ('ferry', np.float64(0.02032461515610029)),
 ('hotel', np.float64(0.019651222903642587)),
 ('day', np.float64(0.016622899721410157)),
 ('kong', np.float64(0.015330582843296867)),
 ('hk', np.float64(0.015315618241245865)),
 ('hong', np.float64(0.015265578158339437)),
 ('hong kong', np.float64(0.015132633289210357)),
 ('time', np.float64(0.014688443841489216)),
 ('travel', np.float64(0.014398745912400347))]

**NOTE**: BERTopic is stocastich which mmeans that the topics might differ across runs. This is mostly due to the stocastisch nature of UMAP.

In [ ]:
### Attributes

## Attributes

There are a number of attributes that you can access after having trained your BERTopic model:


| Attribute | Description |
|------------------------|---------------------------------------------------------------------------------------------|
| topics_               | The topics that are generated for each document after training or updating the topic model. |
| probabilities_ | The probabilities that are generated for each document if HDBSCAN is used. |
| topic_sizes_           | The size of each topic                                                                      |
| topic_mapper_          | A class for tracking topics and their mappings anytime they are merged/reduced.             |
| topic_representations_ | The top *n* terms per topic and their respective c-TF-IDF values.                             |
| c_tf_idf_              | The topic-term matrix as calculated through c-TF-IDF.                                       |
| topic_labels_          | The default labels for each topic.                                                          |
| custom_labels_         | Custom labels for each topic as generated through `.set_topic_labels`.                                                               |
| topic_embeddings_      | The embeddings for each topic if `embedding_model` was used.                                                              |
| representative_docs_   | The representative documents for each topic if HDBSCAN is used.                                                |

For example, to access the predicted topics for the first 10 documents, we simply run the following:

In [ ]:
topic_model.topics_[:10]

# **Visualization**
There are several visualization options available in BERTopic, namely the visualization of topics, probabilities and topics over time. Topic modeling is, to a certain extent, quite subjective. Visualizations help understand the topics that were created.

## Visualize Topics
After having trained our `BERTopic` model, we can iteratively go through perhaps a hundred topic to get a good
understanding of the topics that were extract. However, that takes quite some time and lacks a global representation.
Instead, we can visualize the topics that were generated in a way very similar to
[LDAvis](https://github.com/cpsievert/LDAvis):

In [ ]:
topic_model.visualize_topics()

## Visualize Topic Probabilities

The variable `probabilities` that is returned from `transform()` or `fit_transform()` can
be used to understand how confident BERTopic is that certain topics can be found in a document.

To visualize the distributions, we simply call:

In [ ]:
topic_model.visualize_distribution(probs[200], min_probability=0.015)

## Visualize Topic Hierarchy

The topics that were created can be hierarchically reduced. In order to understand the potential hierarchical structure of the topics, we can use scipy.cluster.hierarchy to create clusters and visualize how they relate to one another. This might help selecting an appropriate nr_topics when reducing the number of topics that you have created.

In [ ]:
topic_model.visualize_hierarchy(top_n_topics=50)

## Visualize Terms

We can visualize the selected terms for a few topics by creating bar charts out of the c-TF-IDF scores for each topic representation. Insights can be gained from the relative c-TF-IDF scores between and within topics. Moreover, you can easily compare topic representations to each other.

In [ ]:
topic_model.visualize_barchart(top_n_topics=5)

## Visualize Topic Similarity
Having generated topic embeddings, through both c-TF-IDF and embeddings, we can create a similarity matrix by simply applying cosine similarities through those topic embeddings. The result will be a matrix indicating how similar certain topics are to each other.

In [ ]:
topic_model.visualize_heatmap(n_clusters=20, width=1000, height=1000)

## Visualize Term Score Decline
Topics are represented by a number of words starting with the best representative word. Each word is represented by a c-TF-IDF score. The higher the score, the more representative a word to the topic is. Since the topic words are sorted by their c-TF-IDF score, the scores slowly decline with each word that is added. At some point adding words to the topic representation only marginally increases the total c-TF-IDF score and would not be beneficial for its representation.

To visualize this effect, we can plot the c-TF-IDF scores for each topic by the term rank of each word. In other words, the position of the words (term rank), where the words with the highest c-TF-IDF score will have a rank of 1, will be put on the x-axis. Whereas the y-axis will be populated by the c-TF-IDF scores. The result is a visualization that shows you the decline of c-TF-IDF score when adding words to the topic representation. It allows you, using the elbow method, the select the best number of words in a topic.


In [ ]:
topic_model.visualize_term_rank()

# **Topic Representation**
After having created the topic model, you might not be satisfied with some of the parameters you have chosen. Fortunately, BERTopic allows you to update the topics after they have been created.

This allows for fine-tuning the model to your specifications and wishes.

## Update Topics
When you have trained a model and viewed the topics and the words that represent them,
you might not be satisfied with the representation. Perhaps you forgot to remove
stopwords or you want to try out a different `n_gram_range`. We can use the function `update_topics` to update
the topic representation with new parameters for `c-TF-IDF`:


In [ ]:
topic_model.update_topics(docs, n_gram_range=(1, 2))

In [ ]:
topic_model.get_topic(0)   # We select topic that we viewed before

[('space', 0.011119596146117955),
 ('nasa', 0.0047697533973351915),
 ('shuttle', 0.0044533985251824495),
 ('orbit', 0.004129278694477752),
 ('spacecraft', 0.004011023125258004),
 ('satellite', 0.003783732360211832),
 ('moon', 0.003639954930862572),
 ('lunar', 0.0034753177228921146),
 ('the moon', 0.002821040122532999),
 ('mars', 0.0028033947303940923)]

## Topic Reduction
We can also reduce the number of topics after having trained a BERTopic model. The advantage of doing so,
is that you can decide the number of topics after knowing how many are actually created. It is difficult to
predict before training your model how many topics that are in your documents and how many will be extracted.
Instead, we can decide afterwards how many topics seems realistic:





In [ ]:
topic_model.reduce_topics(docs, nr_topics=60)

2021-10-17 06:05:02,666 - BERTopic - Reduced number of topics from 220 to 61


In [ ]:
# Access the newly updated topics with:
print(topic_model.topics_)

[0, 32, -1, -1, -1, -1, -1, 0, 0, -1, -1, -1, -1, -1, -1, 14, -1, -1, -1, 4, 6, -1, -1, 4, 0, -1, -1, -1, -1, 20, -1, 48, 5, 0, 25, 11, 24, -1, 4, -1, -1, 23, 51, -1, 0, -1, -1, 7, 1, 5, -1, -1, 48, 1, -1, 4, -1, -1, -1, -1, 0, -1, -1, -1, -1, -1, 0, -1, -1, -1, -1, 19, 35, -1, -1, -1, 0, 37, -1, 0, 54, 6, 58, 51, 30, -1, -1, 19, -1, -1, 0, 2, 45, 32, -1, -1, 21, -1, -1, -1, -1, -1, -1, 3, 2, 0, -1, -1, -1, -1, 11, -1, -1, 15, 14, -1, -1, -1, 0, 5, 5, -1, -1, -1, 9, -1, -1, 2, -1, -1, -1, -1, -1, 0, 55, 2, 21, 1, -1, -1, -1, 11, 31, -1, -1, -1, 11, 10, 0, 7, 4, 4, 55, -1, -1, -1, -1, 3, 8, 16, -1, 2, -1, -1, -1, -1, -1, -1, -1, 25, 24, -1, 28, -1, -1, -1, 0, -1, 4, 0, 6, 0, -1, -1, -1, 2, -1, 31, -1, -1, 5, -1, 2, 13, -1, -1, 14, -1, -1, 10, -1, -1, -1, -1, -1, -1, 18, -1, 53, 13, 4, 44, -1, 5, 4, -1, -1, -1, -1, 2, 0, 34, -1, 6, 1, -1, 1, 4, -1, 0, -1, -1, -1, 40, 0, -1, -1, 0, 0, -1, -1, -1, 25, 39, -1, 7, 2, 6, -1, 29, -1, -1, 40, -1, 23, -1, -1, -1, -1, -1, -1, -1, -1, -1, 32, 21, 

# **Search Topics**
After having trained our model, we can use `find_topics` to search for topics that are similar
to an input search_term. Here, we are going to be searching for topics that closely relate the
search term "vehicle". Then, we extract the most similar topic and check the results:

In [ ]:
similar_topics, similarity = topic_model.find_topics("vehicle", top_n=5); similar_topics

[71, 45, 77, 9, 56]

In [ ]:
topic_model.get_topic(71)

[('car', 0.03740731827314482),
 ('the car', 0.027790363401304377),
 ('dealer', 0.013837911908704722),
 ('the dealer', 0.009515109324321468),
 ('owner', 0.008430722097917726),
 ('previous owner', 0.008157988442865012),
 ('cars', 0.005827046491488879),
 ('the odometer', 0.00514870077683653),
 ('bought car', 0.004667512506960727),
 ('car with', 0.004498685875558186)]

# **Model serialization**
The model and its internal settings can easily be saved. Note that the documents and embeddings will not be saved. However, UMAP and HDBSCAN will be saved.

In [ ]:
# Save model
topic_model.save("my_model")

In [ ]:
# Load model
my_model = BERTopic.load("my_model")

# **Embedding Models**
The parameter `embedding_model` takes in a string pointing to a sentence-transformers model, a SentenceTransformer, or a Flair DocumentEmbedding model.

## Sentence-Transformers
You can select any model from sentence-transformers here and pass it through BERTopic with embedding_model:



In [ ]:
topic_model = BERTopic(embedding_model="xlm-r-bert-base-nli-stsb-mean-tokens")

Or select a SentenceTransformer model with your own parameters:


In [ ]:
from sentence_transformers import SentenceTransformer

sentence_model = SentenceTransformer("distilbert-base-nli-mean-tokens", device="cpu")
topic_model = BERTopic(embedding_model=sentence_model, verbose=True)

Click [here](https://www.sbert.net/docs/pretrained_models.html) for a list of supported sentence transformers models.  
